In [1]:
import numpy as np
import vispy
import vispy.gloo as gloo
from vispy import app
from vispy.util.transforms import perspective, translate, rotate

# load the vispy bindings for the IPython notebook which enables webGL
%load_ext vispy.ipython

Vispy IPython module has loaded successfully

In [2]:
import pyopencl as cl
from pyopencl.tools import get_gl_sharing_context_properties
import sys

In [3]:
n = 100
l = np.linspace(-1,1,100).astype(np.float32)
a_position = np.mgrid[-1:1:5j,-1:1:5j,-1:1:4j].reshape(3,100).T.astype(np.float32)
a_id = np.random.randint(0, 30, (n, 1))
a_id = np.sort(a_id, axis=0).astype(np.float32)

src = """
__kernel void generate_sin(__global float3* a)
{
    int id = get_global_id(0);
    a[id].y = native_sin(a[id.y]);
}
"""

In [4]:
VERT_SHADER = """
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
attribute vec3 a_position;
attribute float a_id;
varying float v_id;
void main (void) {
    v_id = a_id;
    gl_Position = u_projection * u_view * u_model * vec4(a_position,1.0);
}
"""

FRAG_SHADER = """
varying float v_id;
void main()
{
    float f = fract(v_id);
    // The second useless test is needed on OSX 10.8 
    if( (f > 0.0001) && (f < .9999) )
        discard;
    else
        gl_FragColor = vec4(0,0,0,1);
}
"""

In [5]:
class Canvas(app.Canvas):

    # ---------------------------------
    def __init__(self, size=None):

        app.Canvas.__init__(self, keys='interactive', size=size)

        self.program = gloo.Program(VERT_SHADER, FRAG_SHADER)

        # Set uniform and attribute
        self.program['a_id'] = gloo.VertexBuffer(a_id)
        self.program['a_position'] = gloo.VertexBuffer(a_position)
        
        vbo = self.program['a_position'].id

        self.translate = 5
        self.view = translate((0, 0, -self.translate), dtype=np.float32)
        self.model = np.eye(4, dtype=np.float32)

        gloo.set_viewport(0, 0, self.physical_size[0], self.physical_size[1])
        self.projection = perspective(45.0, self.size[0] /
                                      float(self.size[1]), 1.0, 1000.0)
        self.program['u_projection'] = self.projection

        self.program['u_model'] = self.model
        self.program['u_view'] = self.view

        self.theta = 0
        self.phi = 0

        self.context.set_clear_color('white')
        self.context.set_state('translucent')

        self.timer = app.Timer('auto', connect=self.on_timer, start=True)

        platform = cl.get_platforms()[0]
       
        self.show()
    # ---------------------------------
    def on_key_press(self, event):
        if event.text == ' ':
            if self.timer.running:
                self.timer.stop()
            else:
                self.timer.start()

    # ---------------------------------
    def on_timer(self, event):
        self.theta += .5
        self.phi += .5
        self.model = np.dot(rotate(self.theta, (0, 0, 1)),
                            rotate(self.phi, (0, 1, 0)))
        self.program['u_model'] = self.model
        self.update()

    # ---------------------------------
    def on_resize(self, event):
        gloo.set_viewport(0, 0, event.physical_size[0], event.physical_size[1])
        self.projection = perspective(45.0, event.size[0] /
                                      float(event.size[1]), 1.0, 1000.0)
        self.program['u_projection'] = self.projection

    # ---------------------------------
    def on_mouse_wheel(self, event):
        self.translate += event.delta[1]
        self.translate = max(2, self.translate)
        self.view = translate((0, 0, -self.translate))
        self.program['u_view'] = self.view
        self.update()

    # ---------------------------------
    def on_draw(self, event):
        self.context.clear()
        self.program.draw('line_strip')

In [6]:
c = Canvas(size=(300, 300))
#from vispy.app.backends.ipython import VispyWidget
#w = VispyWidget()
#w.set_canvas(c)
#w

  File "/Users/amorg7/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)



In [10]:
c.context.

In [20]:
get_gl_sharing_context_properties()

[(268435456, 0)]

In [14]:
a = gloo.VertexBuffer(a_id)

In [19]:
c.program['a_id'].id

11

In [32]:
gloo.buffer.DataBuffer?

In [9]:
cl.get_platforms()[0]

<pyopencl.Platform 'Apple' at 0x7fff0000>

In [31]:
get_gl_sharing_context_properties()

[(268435456, 0)]

In [25]:
platform = cl.get_platforms()[0]

In [28]:
ctx = cl.Context(properties=[
                    (cl.context_properties.PLATFORM, platform)],
                    devices = [platform.get_devices()[1]])

In [29]:
ctx

<pyopencl.Context at 0x7ff28d701040 on <pyopencl.Device 'Iris Pro' on 'Apple' at 0x1024500>>

In [40]:
from __future__ import absolute_import
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.raw.GL.VERSION.GL_1_5 import glBufferData as rawGlBufferData
import pyopencl as cl

In [ ]:
n_vertices = 10000

src = """
__kernel void generate_sin(__global float2* a)
{
    int id = get_global_id(0);
    int n = get_global_size(0);
    float r = (float)id / (float)n;
    float x = r * 16.0f * 3.1415f;
    a[id].x = r * 2.0f - 1.0f;
    a[id].y = native_sin(x);
}
"""

def initialize():
    platform = cl.get_platforms()[0]

    from pyopencl.tools import get_gl_sharing_context_properties
    import sys
    if sys.platform == "darwin":
        ctx = cl.Context(properties=get_gl_sharing_context_properties(),
                devices=[])
    else:
        # Some OSs prefer clCreateContextFromType, some prefer
        # clCreateContext. Try both.
        try:
            ctx = cl.Context(properties=[
                (cl.context_properties.PLATFORM, platform)]
                + get_gl_sharing_context_properties())
        except:
            ctx = cl.Context(properties=[
                (cl.context_properties.PLATFORM, platform)]
                + get_gl_sharing_context_properties(),
                devices = [platform.get_devices()[0]])

    glClearColor(1, 1, 1, 1)
    glColor(0, 0, 1)
    vbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, vbo)
    rawGlBufferData(GL_ARRAY_BUFFER, n_vertices * 2 * 4, None, GL_STATIC_DRAW)
    glEnableClientState(GL_VERTEX_ARRAY)
    glVertexPointer(2, GL_FLOAT, 0, None)
    coords_dev = cl.GLBuffer(ctx, cl.mem_flags.READ_WRITE, int(vbo))
    prog = cl.Program(ctx, src).build()
    queue = cl.CommandQueue(ctx)
    cl.enqueue_acquire_gl_objects(queue, [coords_dev])
    prog.generate_sin(queue, (n_vertices,), None, coords_dev)
    cl.enqueue_release_gl_objects(queue, [coords_dev])
    queue.finish()
    glFlush()

def display():
    glClear(GL_COLOR_BUFFER_BIT)
    glDrawArrays(GL_LINE_STRIP, 0, n_vertices)
    glFlush()

def reshape(w, h):
    glViewport(0, 0, w, h)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glMatrixMode(GL_MODELVIEW)

glutInitWindowSize(800, 160)
glutInitWindowPosition(0, 0)
glutCreateWindow('OpenCL/OpenGL Interop Tutorial: Sin Generator')
glutDisplayFunc(display)
glutReshapeFunc(reshape)
initialize()
glutMainLoop()

In [ ]:
gloo.gl.

In [11]:
c = app.Canvas(keys='interactive')

In [12]:
vertex = """
attribute vec2 a_position;
void main (void)
{
    gl_Position = vec4(a_position, 0.0, 1.0);
}
"""

In [13]:
fragment = """
void main()
{
    gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
}
"""

In [14]:
program = gloo.Program(vertex, fragment)

In [18]:
program['a_position'] = np.c_[
        np.linspace(-1.0, +1.0, 1000).astype(np.float32),
        np.random.uniform(-0.5, +0.5, 1000).astype(np.float32)]

In [21]:
@c.connect
def on_resize(event):
    gloo.set_viewport(0, 0, *event.size)

In [22]:
@c.connect
def on_draw(event):
    gloo.clear((1,1,1,1))

In [36]:
gloo.gl.

In [35]:
prog = cl.Program(c.context, src).build()

AttributeError: 'GLContext' object has no attribute 'devices'

In [37]:
app.create()

In [ ]:
app.

In [ ]:
c.show()
app.run()